# RAG with Azure AI search

In [1]:
# Import required libraries
import os
import json
from dotenv import load_dotenv

from tenacity import retry, wait_random_exponential, stop_after_attempt
from openai import AzureOpenAI
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
    SearchIndex,
    AzureOpenAIVectorizer,
    AzureOpenAIParameters
)


from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import json

load_dotenv()

True

In [17]:
# Configure environment variables
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")
key = os.getenv("AZURE_SEARCH_ADMIN_KEY")

AZURE_OPENAI_EMBEDDINGS_ADA_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_EMBEDDINGS_ADA_DEPLOYMENT_NAME")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION="2024-02-01"
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AISTUDIO_OPENAI_GPT4_DEPLOYMENT_NAME = os.getenv("AISTUDIO_OPENAI_GPT4_DEPLOYMENT_NAME")
azure_openai_embedding_dimensions = 1536
index_name = "books"

In [15]:
# Configure OpenAI API
aoai_client = AzureOpenAI(
  azure_endpoint = AZURE_OPENAI_ENDPOINT, 
  api_key=AZURE_OPENAI_API_KEY,  
  api_version=AZURE_OPENAI_API_VERSION
)


In [23]:
from azure.search.documents.models import VectorizedQuery
credential = AzureKeyCredential(key)
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)

# Generate Document Embeddings using OpenAI Ada Model
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Function to generate embeddings for title and content fields, also used for query embeddings
def calc_embeddings(text):
    # model = "deployment_name"
    embeddings = aoai_client.embeddings.create(input = [text], model=AZURE_OPENAI_EMBEDDINGS_ADA_DEPLOYMENT_NAME).data[0].embedding
    return embeddings

def do_search(query):
    fields = "embedding"
    embedding = calc_embeddings(query)
    vector_query = VectorizedQuery(vector=embedding, k_nearest_neighbors=3, fields=fields)
  
    results = search_client.search(  
        search_text=None,  
        vector_queries= [vector_query],
        select=["content"],
    )  
    answer = ''
    for result in results:  
        print(f"Score: {result['@search.score']}")  
        print(f"Content: {result['content']}")  
        answer = answer + result['content']
    return answer

In [27]:
question = "Why does the coffin prepared for Queequeg become Ishmael's life buoy once the Pequod sinks?"
answers = do_search(question)

Score: 0.878709
Content: the studded iron-bound cask followed the sailor to the bottom, as if to 
yield him his pillow, though in sooth but a hard one. 
 
And thus the first man of the Pequod that mounted the mast to look out 
for the White Whale, on the White Whale’s own peculiar ground; that man 
was swallowed up in the deep. But few, perhaps, thought of that at the 
time. Indeed, in some sort, they were not grieved at this event, at 
least as a portent; for they regarded it, not as a foreshadowing of 
evil in the future, but as the fulfilment of an evil already presaged. 
They declared that now they knew the reason of those wild shrieks they 
had heard the night before. But again the old Manxman said nay. 
 
The lost life-buoy was now to be replaced; Starbuck was directed to see 
to it; but as no cask of sufficient lightness could be found, and as in 
the feverish eagerness of what seemed the approaching crisis of the 
voyage, all hands were impatient of any toil but what was direct

In [18]:
def call_openAI(text):
    response = aoai_client.chat.completions.create(
        model=AISTUDIO_OPENAI_GPT4_DEPLOYMENT_NAME,
        messages = text,
        temperature=0.7,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    return response.choices[0].message.content

In [ ]:
# This prompt provides instructions to the model. 
# The prompt includes the query and the source, which are specified further down in the code.
grounded_prompt="""
You are a friendly assistant answering users questions.
Answer the query using only the answers provided below in a friendly and concise bulleted manner.
Answer ONLY with the facts listed in the list of answers below.
If there isn't enough information below, say you don't know.
Do not generate answers that don't use the answers below.
Query: {question}
Sources:\n{answers}
"""
# prepare prompt
messages=[
    {
        "role": "user",
        "content": grounded_prompt.format(question=question, answers=answers)
    }
]
  
result = call_openAI(messages)
display(result)

'- The lost life-buoy was to be replaced, but no sufficiently light cask could be found.\n- Queequeg hinted his coffin could be used as a life-buoy.\n- Starbuck directed the carpenter to rig the coffin as a life-buoy.\n- The coffin was prepared and used as a life-buoy because there was no other option available.'